In [1]:
!conda install -c conda-forge folium=0.5.0
!pip install geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
!pip install beautifulsoup4
!pip install geopy

In [3]:
!pip install lxml

### Import required libraries

In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

## Bengaluru Data

### Using BeautifulSoup to parse html file

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore'
html_file = requests.get(url).text

In [6]:
soup = BeautifulSoup(html_file,'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of neighbourhoods in Bangalore - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Bangalore","wgTitle":"List of neighbourhoods in Bangalore","wgCurRevisionId":896689983,"wgRevisionId":896689983,"wgArticleId":53117184,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from August 2017","Use Indian English from August 2017","All Wikipedia articles written in Indian English","Neighbourhoods in Bangalore","Bangalore-related lists","Lists of neighbourhoods in Indian cities"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTa

### Reading required data

In [7]:
Region = []
for reg in soup.find_all('span',class_ = 'mw-headline'):
        Region.append(reg.text)
del Region[8:10]
print(Region)

['Central', 'Eastern', 'North-Eastern', 'Northern', 'South-Eastern', 'Southern', 'Southern suburbs', 'Western']


In [8]:
dfreg = []
dfarea = []
i = 0
for area1 in soup.find_all('table',class_='wikitable sortable'):
    for area2 in area1.find_all('td'):
        dfarea.append(area2.text.strip())
        dfreg.append(Region[i])
    i = i+1
dfreg = dfreg[::3]        
dfarea = dfarea[::3]
print(len(dfreg))
print(len(dfarea))

65
65


In [9]:
dflist = ['Region','Neighborhood']
df = pd.DataFrame(columns = dflist)
df['Region'] = dfreg
df['Neighborhood'] = dfarea
df

,Region,Neighborhood
0,Central,Cantonment area
1,Central,Domlur
2,Central,Indiranagar
3,Central,Jeevanbheemanagar
4,Central,Malleswaram
...,...,...
60,Western,Nandini Layout
61,Western,Nayandahalli
62,Western,Rajajinagar
63,Western,Rajarajeshwari Nagar


### Get Bengaluru Neighborhood coordinates

In [10]:
Head = ['Latitude', 'Longitude']
beng_csv = 'Bangalore_Coordinates.csv'
bengaluru_coord = pd.read_csv(beng_csv, names = Head)
bengaluru_coord.head()

,Latitude,Longitude
0,13.027925,77.533676
1,12.962467,77.638196
2,12.973291,77.640467
3,12.966008,77.657672
4,13.016341,77.558664


### Join Bengaluru Neighborhood data with their coordinates

In [11]:
dff = df.join(bengaluru_coord)
dff

,Region,Neighborhood,Latitude,Longitude
0,Central,Cantonment area,13.027925,77.533676
1,Central,Domlur,12.962467,77.638196
2,Central,Indiranagar,12.973291,77.640467
3,Central,Jeevanbheemanagar,12.966008,77.657672
4,Central,Malleswaram,13.016341,77.558664
...,...,...,...,...
60,Western,Nandini Layout,13.010406,77.537803
61,Western,Nayandahalli,12.944600,77.525600
62,Western,Rajajinagar,12.988234,77.554883
63,Western,Rajarajeshwari Nagar,12.927428,77.515372


### Get coordinates of Bengaluru

In [12]:
address = 'Bengaluru, Karnataka'

geolocator = Nominatim(user_agent="Beng")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru are 12.9791198, 77.5912997.


### Visualize map of Bengaluru with Neighborhood markers

In [13]:
import folium
# create map of Bengaluru using latitude and longitude values
map_bengaluru = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dff['Latitude'], dff['Longitude'], dff['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bengaluru)  
    
map_bengaluru

### Foursquare API Credentials

In [14]:
CLIENT_ID = 'OREU0ESXHHUMVDEFEGAJAEVFTCQRUR2EZJWZYLBUM1A5YCUY' # your Foursquare ID
CLIENT_SECRET = '1PDNS3NFJ4UCMMFUXHADFBEST54XSFWZWT4P1QJSJ55XPRER' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OREU0ESXHHUMVDEFEGAJAEVFTCQRUR2EZJWZYLBUM1A5YCUY
CLIENT_SECRET:1PDNS3NFJ4UCMMFUXHADFBEST54XSFWZWT4P1QJSJ55XPRER


### Defining function to obtain nearby venues

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Getting nearby venues in Bengaluru

In [16]:
bengaluru_venues = getNearbyVenues(names=dff['Neighborhood'],
                                   latitudes=dff['Latitude'],
                                   longitudes=dff['Longitude']
                                  )

Cantonment area
Domlur
Indiranagar
Jeevanbheemanagar
Malleswaram
Pete area
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
CV Raman Nagar
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
R. T. Nagar
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


### Let us remove venue categories containing 'Neighborhood'

In [17]:
bengaluru_venues = bengaluru_venues[bengaluru_venues['Venue Category'] != 'Neighborhood']
print(bengaluru_venues.shape)

(2807, 7)


### Display number of Unique Categories

In [18]:
print('There are {} uniques categories.'.format(len(bengaluru_venues['Venue Category'].unique())))

There are 210 uniques categories.


### Display count of Unique Categories in each Neighborhood

In [19]:
bengaluru_venues.groupby('Neighborhood')['Venue Category'].nunique().to_frame().sort_values('Venue Category', ascending = False)

,Venue Category
Neighborhood,
Indiranagar,51
Domlur,48
Shivajinagar,47
Seshadripuram,47
Koramangala,46
...,...
Anjanapura,7
Horamavu,6
Kamakshipalya,6


### Display count of each Venue Category in each Neighborhood

In [20]:
vencount = bengaluru_venues.groupby('Neighborhood')['Venue Category'].value_counts().to_frame()
vencount.columns = ['Venue Count']
vencount.sort_values('Venue Count', ascending = False)
vencount

Venue Count
Neighborhood Venue Category                 
Anjanapura   ATM                           1
             Asian Restaurant              1
             Campground                    1
             Flower Shop                   1
             Photography Lab               1
...                                      ...
Yeshwanthpur Park                          1
             Seafood Restaurant            1
             Shopping Mall                 1
             Spa                           1
             Wine Bar                      1

[1589 rows x 1 columns]

## Delhi Data

### Load Delhi Neighborhood Data

In [21]:
delhi_head = ['Region','Neighborhood']
delhi_data = pd.read_csv('Delhi_Neighbourhoods.csv', names = delhi_head)
delhi_data.head()

,Region,Neighborhood
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Azadpur
3,North West Delhi,Bawana
4,North West Delhi,Begum Pur


In [22]:
delhi_coord = pd.read_csv('Delhi_Coord.csv', names = Head)
delhi_coord.head()

,Latitude,Longitude
0,28.614193,77.071541
1,28.698510,77.185030
2,28.707657,77.175547
3,28.799660,77.032885
4,28.722400,77.066150


In [23]:
delhi_data = delhi_data.join(delhi_coord)

In [24]:
delhi_data.shape

(185, 4)

In [25]:
delhi_data['Neighborhood'].nunique()

175

### Remove redundant rows

In [26]:
delhi_data.drop_duplicates(subset ="Neighborhood", 
                     keep = 'first', inplace = True)
delhi_data.reset_index(drop = True, inplace = True)
delhi_data

,Region,Neighborhood,Latitude,Longitude
0,North West Delhi,Adarsh Nagar,28.614193,77.071541
1,North West Delhi,Ashok Vihar,28.698510,77.185030
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Begum Pur,28.722400,77.066150
...,...,...,...,...
170,West Delhi,Tihar Village,28.634636,77.107112
171,West Delhi,Tilak Nagar,28.639650,77.094039
172,West Delhi,Uttam Nagar,28.629860,77.053080
173,West Delhi,Vikas Nagar,28.644009,77.054470


### Get coordinates of Delhi

In [27]:
address_delhi = 'Delhi, India'

geolocator = Nominatim(user_agent="del")
location_delhi = geolocator.geocode(address_delhi)
latitude_delhi = location_delhi.latitude
longitude_delhi = location_delhi.longitude
print('The geograpical coordinates of Delhi are {}, {}.'.format(latitude_delhi, longitude_delhi))

The geograpical coordinates of Delhi are 28.6517178, 77.2219388.


### Visualize map of Delhi with Neighborhood markers

In [28]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude_delhi, longitude_delhi], zoom_start=11)

# add markers to map
for lat, lng, label2 in zip(delhi_data['Latitude'], delhi_data['Longitude'], delhi_data['Neighborhood']):
    label2 = folium.Popup(label2, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label2,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
    
map_delhi

### Getting nearby venues in Delhi

In [ ]:
delhi_venues = getNearbyVenues(names=delhi_data['Neighborhood'],
                                   latitudes=delhi_data['Latitude'],
                                   longitudes=delhi_data['Longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Begum Pur
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Rohini Sub City
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Ghantewala
Gulabi Bagh
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sadar Bazaar
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shastri Nagar
Tees Hazari
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Nand Nagri
Naveen Shahdara
New Usmanpur
Sadatpur
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Daryaganj
Jhandewalan
Karol Bagh
Rajender Nagar
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Laxmibai Nagar
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar


### Let us remove venue categories containing 'Neighborhood'

In [ ]:
delhi_venues = delhi_venues[delhi_venues['Venue Category'] != 'Neighborhood']
print(delhi_venues.shape)

### Display number of Unique Categories

In [ ]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

### Display count of Unique Categories in each Neighborhood

In [ ]:
delhi_venues.groupby('Neighborhood')['Venue Category'].nunique().to_frame().sort_values('Venue Category', ascending = False)

### Display count of each Venue Category in each Neighborhood

In [ ]:
vencount2 = delhi_venues.groupby('Neighborhood')['Venue Category'].value_counts().to_frame()
vencount2.columns = ['Venue Count']
vencount2.sort_values('Venue Count', ascending = False)
vencount2

## Join Delhi and Bengaluru Data

In [ ]:
del_ban_data = bengaluru_venues.append(delhi_venues, ignore_index = True) 
del_ban_data

### One hot encoding

In [ ]:
# one hot encoding
del_ban_onehot = pd.get_dummies(del_ban_data[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
del_ban_onehot['Neighborhood'] = del_ban_data['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [del_ban_onehot.columns[-1]] + list(del_ban_onehot.columns[:-1])
del_ban_onehot = del_ban_onehot[fixed_columns]

del_ban_onehot.head()

### Count of Venue Category in each Neighborhood

In [ ]:
count_data = del_ban_onehot.groupby(['Neighborhood']).sum().reset_index()
count_data.head()

### Grouping Data based on Neighborhood

In [ ]:
del_ban_grouped = del_ban_onehot.groupby('Neighborhood').mean().reset_index()
del_ban_grouped

## Clustering Data using K-Means

In [ ]:
del_ban_clustering = del_ban_grouped.drop('Neighborhood', 1)
del_ban_clustering.head()

### Finding optimal 'K'

In [ ]:
distortions = []
K = range(1,9)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0).fit(del_ban_clustering)
    distortions.append(kmeanModel.inertia_)

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of Squared Error')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:

for n in range(2,9):
    kmeans = KMeans(n_clusters=n).fit(del_ban_clustering)
    label = kmeans.labels_
    sil_coeff = silhouette_score(del_ban_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n, sil_coeff))

### Apply K-Means for K=7

In [ ]:
# set number of clusters
cluster_num = 7
# run k-means clustering
kmeans_cluster = KMeans(n_clusters=cluster_num).fit(del_ban_clustering)
# check cluster labels
kmeans_cluster.labels_[0:10]

### Display top 10 Venues of Each Neighborhood with their Cluster Labels

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = del_ban_grouped['Neighborhood']

for ind in np.arange(del_ban_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(del_ban_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.insert(1, 'Cluster Label', kmeans_cluster.labels_)
neighborhoods_venues_sorted.head()

### Add City and Region to each Neighborhood

In [ ]:
banreg = dff[['Region','Neighborhood']]
banreg.insert(0,'City','Bengaluru')
delreg = delhi_data[['Region','Neighborhood']]
delreg.insert(0,'City','Delhi')
reg_data = banreg.append(delreg)
reg_data

In [ ]:
neighborhood = neighborhoods_venues_sorted.set_index('Neighborhood').join(reg_data.set_index('Neighborhood')).reset_index()
neighborhood.head()

### Move Columns 'Neighborhood and 'City' to first position

In [ ]:
cols = neighborhood.columns.tolist()
cols = cols[-2:] + cols[:-2]
neighborhood = neighborhood[cols]
neighborhood.head()

## Group 1

In [ ]:
group1_common = neighborhood.loc[neighborhood['Cluster Label'] == 0, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group1 = group1_common[['City','Region','Neighborhood']].reset_index(drop=True)
group1.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
Cat_col = ['Category','Count']
group1_venues = group1.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group1_venues = group1_venues.mean().to_frame().reset_index()
group1_venues.columns = Cat_col
group1_venues = group1_venues.sort_values('Count', ascending = False)
group1_venues = group1_venues[group1_venues['Count'] != 0].reset_index(drop = True)
group1_venues

In [ ]:
group1_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 1 - Top 20 venues',size=15) 

plt.show()

## Group 2

In [ ]:
group2_common = neighborhood.loc[neighborhood['Cluster Label'] == 1, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group2 = group2_common[['City','Region','Neighborhood']].reset_index(drop=True)
group2.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group2_venues = group2.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group2_venues = group2_venues.mean().to_frame().reset_index()
group2_venues.columns = Cat_col
group2_venues = group2_venues.sort_values('Count', ascending = False)
group2_venues = group2_venues[group2_venues['Count'] != 0].reset_index(drop = True)
group2_venues

In [ ]:
group2_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 2 - Top 20 venues',size=15) 

plt.show()

## Group 3

In [ ]:
group3_common = neighborhood.loc[neighborhood['Cluster Label'] == 2, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group3 = group3_common[['City','Region','Neighborhood']].reset_index(drop=True)
group3.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group3_venues = group3.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group3_venues = group3_venues.mean().to_frame().reset_index()
group3_venues.columns = Cat_col
group3_venues = group3_venues.sort_values('Count', ascending = False)
group3_venues = group3_venues[group3_venues['Count'] != 0].reset_index(drop = True)
group3_venues

In [ ]:
group3_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 3 - Top 20 venues',size=15) 

plt.show()

## Group 4

In [ ]:
group4_common = neighborhood.loc[neighborhood['Cluster Label'] == 3, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group4 = group4_common[['City','Region','Neighborhood']].reset_index(drop=True)
group4.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group4_venues= group4.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group4_venues = group4_venues.mean().to_frame().reset_index()
group4_venues.columns = Cat_col
group4_venues = group4_venues.sort_values('Count', ascending = False)
group4_venues = group4_venues[group4_venues['Count'] != 0].reset_index(drop = True)
group4_venues

In [ ]:
group4_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 4 - Top 20 venues',size=15) 

plt.show()

## Group 5

In [ ]:
group5_common = neighborhood.loc[neighborhood['Cluster Label'] == 4, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group5 = group5_common[['City','Region','Neighborhood']].reset_index(drop=True)
group5.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group5_venues = group5.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group5_venues = group5_venues.mean().to_frame().reset_index()
group5_venues.columns = Cat_col
group5_venues = group5_venues.sort_values('Count', ascending = False)
group5_venues = group5_venues[group5_venues['Count'] != 0].reset_index(drop = True)
group5_venues

In [ ]:
group5_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 5 - Top 20 venues',size=15) 

plt.show()

## Group 6

In [ ]:
group6_common = neighborhood.loc[neighborhood['Cluster Label'] == 5, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group6 = group6_common[['City','Region','Neighborhood']].reset_index(drop=True)
group6.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group6_venues = group6.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group6_venues = group6_venues.mean().to_frame().reset_index()
group6_venues.columns = Cat_col
group6_venues = group6_venues.sort_values('Count', ascending = False)
group6_venues = group6_venues[group6_venues['Count'] != 0].reset_index(drop = True)
group6_venues

In [ ]:
group6_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 6 - Top 20 venues',size=15) 

plt.show()

## Group 7

In [ ]:
group7_common = neighborhood.loc[neighborhood['Cluster Label'] == 6, neighborhood.columns[list(range(0, neighborhood.shape[1]))]]
group7 = group7_common[['City','Region','Neighborhood']].reset_index(drop=True)
group7.sort_values('City').reset_index(drop = True)

### Average Count of Venue Category

In [ ]:
group7_venues = group7.set_index('Neighborhood').join(count_data.set_index('Neighborhood')).reset_index()
group7_venues = group7_venues.mean().to_frame().reset_index()
group7_venues.columns = Cat_col
group7_venues = group7_venues.sort_values('Count', ascending = False)
group7_venues = group7_venues[group7_venues['Count'] != 0].reset_index(drop = True)
group7_venues

In [ ]:
group7_venues.set_index('Category').head(20).plot(kind='barh', figsize=(17, 10))

plt.xlabel('Mean Count',size=12) 
plt.ylabel('Categories',size=12)
plt.title('Group 7 - Top 20 venues',size=15) 

plt.show()